In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install openl3 soundfile

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.4/323.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openl3: filename=openl3-0.4.2-py2.py3-none-any.whl size=249327028 sha256=528b061e3e03121ae701c5ca82addf3d44de2244293f982e24a21c8d7da5cca4
  Stored in directory: /root/.cache/pip/wheels/d0/4d/0a/e57b1dc8ead91b3c5709d9de4f02d1cdd3a91f609a8f1c1062
  Created wheel for kapre: filename=kapre-0.3.7-py3-none-any.whl size=29603 sha256=b4b6b4449d6daee454548f4a4cd72a1d97ad6bd408b962ec507958730b5e10b5
  Stored in directory: /root/.cache/pip/wheels/3e/54/f9/37e9f36590a5431dc207f7dc0eb2e607ff8958d5728e45ef58
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320704 sha256=d7ef58bc0637d8fc70ccdc9c4e37194d2caa7695260229cefe0da62fb1c5bcc3
  Stored in directory: /root

In [3]:
!pip install tensorflow==2.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 37.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  At

In [1]:
import os
import pandas as pd
import librosa
import openl3
import numpy as np
import soundfile as sf

# 경로 설정
AUDIO_DIR = "/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen"
METADATA_FILE = "/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/processed_metadata.csv"
OUTPUT_METADATA_FILE = "/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/extended_metadata.csv"

# OpenL3 태그 추출
def extract_openl3_embeddings(audio_file):
    """
    OpenL3를 사용하여 오디오 파일에서 임베딩을 추출합니다.
    """
    # 오디오 로드
    audio, sr = sf.read(audio_file)

    # OpenL3 요구사항에 따라 16kHz로 리샘플링
    if sr != 16000:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        sr = 16000

    # OpenL3 임베딩 추출
    embeddings, _ = openl3.get_audio_embedding(
        audio, sr, input_repr="mel256", content_type="music", embedding_size=512
    )

    # 임베딩 평균값 사용
    mean_embedding = embeddings.mean(axis=0)
    return mean_embedding

# RMS/BPM 계산
def calculate_audio_features(audio_file):
    """
    오디오 파일에서 RMS(음량)와 BPM(템포)을 계산합니다.
    """
    y, sr = librosa.load(audio_file, sr=None)
    rms = librosa.feature.rms(y=y).mean()
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    return rms, tempo

# 수면음악 조건 분류
def classify_sleep_music(rms, tempo):
    """
    RMS와 템포를 기반으로 수면음악 여부를 분류합니다.
    """
    is_loud = rms > 0.05  # 예: RMS 0.05 이상은 시끄럽다고 판단
    is_fast = tempo > 90  # 예: BPM 90 이상은 빠른 템포로 간주
    if is_loud or is_fast:
        return "수면음악 아님"
    return "수면음악"

# 메타데이터 업데이트 함수
def update_metadata_with_features_and_embeddings(audio_dir, metadata_file, output_metadata_file):
    """
    기존 메타데이터에 OpenL3 임베딩 및 추가 특성을 포함한 정보를 추가합니다.
    """
    # 메타데이터 읽기
    metadata = pd.read_csv(metadata_file)

    # 새 컬럼 추가
    metadata["OpenL3_Embedding"] = ""
    metadata["RMS"] = ""
    metadata["Tempo"] = ""
    metadata["Sleep_Music"] = ""

    # 각 오디오 파일 처리
    for index, row in metadata.iterrows():
        audio_file = os.path.join(audio_dir, row['audio'])  # 'audio'은 메타데이터의 컬럼 이름
        if not os.path.exists(audio_file):
            print(f"파일이 존재하지 않습니다: {audio_file}")
            continue

        try:
            # OpenL3 임베딩 추출
            embedding = extract_openl3_embeddings(audio_file)

            # RMS/BPM 계산
            rms, tempo = calculate_audio_features(audio_file)

            # 수면 음악 분류
            sleep_music_status = classify_sleep_music(rms, tempo)

            # 메타데이터 업데이트
            metadata.at[index, "OpenL3_Embedding"] = np.array2string(embedding, separator=",")
            metadata.at[index, "RMS"] = rms
            metadata.at[index, "Tempo"] = tempo
            metadata.at[index, "Sleep_Music"] = sleep_music_status

            print(f"{audio_file}: 임베딩 및 특성 추출 완료, 수면음악: {sleep_music_status}")
        except Exception as e:
            print(f"처리 실패: {audio_file}, 오류: {e}")

    # 확장된 메타데이터 저장
    metadata.to_csv(output_metadata_file, index=False)
    print(f"확장된 메타데이터가 저장되었습니다: {output_metadata_file}")

# 실행
update_metadata_with_features_and_embeddings(AUDIO_DIR, METADATA_FILE, OUTPUT_METADATA_FILE)

3/3 [==============================] - 48s 16s/step
/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.wav: 임베딩 및 특성 추출 완료, 수면음악: 수면음악
3/3 [==============================] - 48s 15s/step
/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.wav: 임베딩 및 특성 추출 완료, 수면음악: 수면음악 아님
3/3 [==============================] - 47s 16s/step
/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.wav: 임베딩 및 특성 추출 완료, 수면음악: 수면음악 아님
3/3 [==============================] - 46s 15s/step
/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AUDIO_21_R1_2004_01_Track01_wav.wav: 임베딩 및 특성 추출 완료, 수면음악: 수면음악


3/3 [==============================] - 45s 15s/step
/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_04_Track04_wav.wav: 임베딩 및 특성 추출 완료, 수면음악: 수면음악 아님


3/3 [==============================] - 46s 15s/step
/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2009/MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MID--AUDIO_07_R1_2009_07_R1_2009_04_WAV.wav: 임베딩 및 특성 추출 완료, 수면음악: 수면음악 아님
3/3 [==============================] - 47s 16s/step
/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_07_WAV.wav: 임베딩 및 특성 추출 완료, 수면음악: 수면음악
3/3 [==============================] - 46s 15s/step
/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_R2_2013_wav--3.wav: 임베딩 및 특성 추출 완료, 수면음악: 수면음악 아님
3/3 [==============================] - 53s 19s/step
/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_04_WAV.wav: 임베딩 및 특성 추출 완료, 수면음악: 수면음악 아님
3/3 [==============================] - 46s 15s/step
/c

KeyboardInterrupt: 

In [7]:
!pip install musicnn --no-deps

  Using cached musicnn-0.1.0-py3-none-any.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 36.9 MB/s eta 0:00:00


In [9]:
import os
import pandas as pd
import librosa
from musicnn.extractor import extractor

# 경로 설정
AUDIO_DIR = "/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen"
METADATA_FILE = "/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/processed_metadata.csv"
OUTPUT_METADATA_FILE = "/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/extended_metadata.csv"

# Musicnn을 사용한 태깅
def extract_tags(audio_file):
    """
    음악 파일에서 태그를 예측합니다.
    """
    tags, tag_confidence = extractor(audio_file, model='MTT_musicnn')
    return tags, tag_confidence

# RMS/BPM 계산
def calculate_audio_features(audio_file):
    """
    오디오 파일에서 RMS(음량)와 BPM(템포)을 계산합니다.
    """
    y, sr = librosa.load(audio_file, sr=None)
    rms = librosa.feature.rms(y=y).mean()
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    return rms, tempo

# 수면음악 조건 분류
def classify_sleep_music(rms, tempo):
    """
    RMS와 템포를 기반으로 수면음악 여부를 분류합니다.
    """
    is_loud = rms > 0.05  # 예: RMS 0.05 이상은 시끄럽다고 판단
    is_fast = tempo > 90  # 예: BPM 90 이상은 빠른 템포로 간주
    if is_loud or is_fast:
        return "수면음악 아님"
    return "수면음악"

# 감정 태그 추가
def map_emotion_to_tags(tags):
    """
    Musicnn 태그를 기반으로 감정 태그를 추가합니다.
    """
    calm_tags = {"relaxing", "calm", "peaceful"}
    energetic_tags = {"exciting", "energetic", "intense"}

    emotion = "neutral"
    if calm_tags.intersection(tags):
        emotion = "calm"
    elif energetic_tags.intersection(tags):
        emotion = "energetic"

    return f"감정: {emotion}"

# 메타데이터 업데이트 함수
def update_metadata_with_features_and_tags(audio_dir, metadata_file, output_metadata_file):
    """
    기존 메타데이터에 태그 및 추가 특성을 포함한 정보를 추가합니다.
    """
    # 메타데이터 읽기
    metadata = pd.read_csv(metadata_file)

    # 태그 정보를 저장할 컬럼 추가
    metadata["Tags"] = ""
    metadata["Tag_Confidence"] = ""
    metadata["RMS"] = ""
    metadata["Tempo"] = ""
    metadata["Sleep_Music"] = ""
    metadata["Emotion_Tag"] = ""

    # 각 오디오 파일 처리
    for index, row in metadata.iterrows():
        audio_file = os.path.join(audio_dir, row['audio'])  # 'audio'은 메타데이터의 컬럼 이름
        if not os.path.exists(audio_file):
            print(f"파일이 존재하지 않습니다: {audio_file}")
            continue

        try:
            # 태그 추출
            tags, tag_confidence = extract_tags(audio_file)

            # RMS/BPM 계산
            rms, tempo = calculate_audio_features(audio_file)

            # 수면 음악 분류
            sleep_music_status = classify_sleep_music(rms, tempo)

            # 감정 태그 매핑
            emotion_tag = map_emotion_to_tags(set(tags))

            # 메타데이터 업데이트
            metadata.at[index, "Tags"] = ", ".join(tags)
            metadata.at[index, "Tag_Confidence"] = ", ".join([f"{tag}:{conf:.2f}" for tag, conf in tag_confidence.items()])
            metadata.at[index, "RMS"] = rms
            metadata.at[index, "Tempo"] = tempo
            metadata.at[index, "Sleep_Music"] = sleep_music_status
            metadata.at[index, "Emotion_Tag"] = emotion_tag

            print(f"{audio_file}: 태깅 완료 - {tags}, 수면음악: {sleep_music_status}, 감정: {emotion_tag}")
        except Exception as e:
            print(f"처리 실패: {audio_file}, 오류: {e}")

    # 확장된 메타데이터 저장
    metadata.to_csv(output_metadata_file, index=False)
    print(f"확장된 메타데이터가 저장되었습니다: {output_metadata_file}")

# 실행
update_metadata_with_features_and_tags(AUDIO_DIR, METADATA_FILE, OUTPUT_METADATA_FILE)

처리 실패: /content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.wav, 오류: `batch_normalization` is not available with Keras 3.
처리 실패: /content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.wav, 오류: `batch_normalization` is not available with Keras 3.
처리 실패: /content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.wav, 오류: `batch_normalization` is not available with Keras 3.
처리 실패: /content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AUDIO_21_R1_2004_01_Track01_wav.wav, 오류: `batch_normalization` is not available with Keras 3.
처리 실패: /content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2

In [4]:
# 기존 패키지 제거
!pip uninstall -y numpy librosa

# 호환되는 버전 설치
!pip install numpy==1.16.6 librosa==0.8.0 tensorflow==2.5.0

# musicnn 설치
!pip install musicnn==0.0.1

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: librosa 0.8.1
Uninstalling librosa-0.8.1:
  Successfully uninstalled librosa-0.8.1
  Using cached numpy-1.16.6.zip (5.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached librosa-0.8.0.tar.gz (183 kB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement tensorflow==2.5.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No

In [12]:
import os
import pandas as pd
import librosa
from musicnn.extractor import extractor

# 경로 설정
AUDIO_DIR = "/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen"
METADATA_FILE = "/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/processed_metadata.csv"
OUTPUT_METADATA_FILE = "/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/extended_metadata.csv"

# Musicnn을 사용한 태깅
def extract_tags(audio_file):
    """
    음악 파일에서 태그를 예측합니다.
    """
    tags, tag_confidence = extractor(audio_file, model='MTT_musicnn')
    return tags, tag_confidence

# RMS/BPM 계산
def calculate_audio_features(audio_file):
    """
    오디오 파일에서 RMS(음량)와 BPM(템포)을 계산합니다.
    """
    y, sr = librosa.load(audio_file, sr=None)
    rms = librosa.feature.rms(y=y).mean()
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    return rms, tempo

# 수면음악 조건 분류
def classify_sleep_music(rms, tempo):
    """
    RMS와 템포를 기반으로 수면음악 여부를 분류합니다.
    """
    is_loud = rms > 0.05  # 예: RMS 0.05 이상은 시끄럽다고 판단
    is_fast = tempo > 90  # 예: BPM 90 이상은 빠른 템포로 간주
    if is_loud or is_fast:
        return "수면음악 아님"
    return "수면음악"

# 감정 태그 추가
def map_emotion_to_tags(tags):
    """
    Musicnn 태그를 기반으로 감정 태그를 추가합니다.
    """
    calm_tags = {"relaxing", "calm", "peaceful"}
    energetic_tags = {"exciting", "energetic", "intense"}

    emotion = "neutral"
    if calm_tags.intersection(tags):
        emotion = "calm"
    elif energetic_tags.intersection(tags):
        emotion = "energetic"

    return f"감정: {emotion}"

# 메타데이터 업데이트 함수
def update_metadata_with_features_and_tags(audio_dir, metadata_file, output_metadata_file):
    """
    기존 메타데이터에 태그 및 추가 특성을 포함한 정보를 추가합니다.
    """
    # 메타데이터 읽기
    try:
        metadata = pd.read_csv(metadata_file)
        print(f"메타데이터 로드 성공: {metadata_file}")
    except Exception as e:
        print(f"메타데이터 로드 실패: {metadata_file}, 오류: {e}")
        return

    # 태그 정보를 저장할 컬럼 추가
    metadata["Tags"] = ""
    metadata["Tag_Confidence"] = ""
    metadata["RMS"] = ""
    metadata["Tempo"] = ""
    metadata["Sleep_Music"] = ""
    metadata["Emotion_Tag"] = ""

    # 각 오디오 파일 처리
    for index, row in metadata.iterrows():
        try:
            # 오디오 파일 경로 생성
            audio_file = os.path.join(audio_dir, row['audio'])  # 'audio' 컬럼 확인
            print(f"처리 중인 파일 경로: {audio_file}")

            if not os.path.exists(audio_file):
                print(f"파일이 존재하지 않습니다: {audio_file}")
                continue

            # Librosa 파일 로드 테스트
            try:
                y, sr = librosa.load(audio_file, sr=None)
                print(f"Librosa 파일 로드 성공: {audio_file}, 샘플링 레이트: {sr}")
            except Exception as e:
                print(f"Librosa 파일 로드 실패: {audio_file}, 오류: {e}")
                continue

            # Musicnn 태깅 테스트
            try:
                tags, tag_confidence = extract_tags(audio_file)
                print(f"Musicnn 태깅 성공: {tags}")
            except Exception as e:
                print(f"Musicnn 태깅 실패: {audio_file}, 오류: {e}")
                continue

            # RMS/BPM 계산 테스트
            try:
                rms, tempo = calculate_audio_features(audio_file)
                print(f"RMS: {rms}, 템포: {tempo}")
            except Exception as e:
                print(f"RMS/BPM 계산 실패: {audio_file}, 오류: {e}")
                continue

            # 수면 음악 분류
            sleep_music_status = classify_sleep_music(rms, tempo)

            # 감정 태그 매핑
            emotion_tag = map_emotion_to_tags(set(tags))

            # 메타데이터 업데이트
            metadata.at[index, "Tags"] = ", ".join(tags)
            metadata.at[index, "Tag_Confidence"] = ", ".join([f"{tag}:{conf:.2f}" for tag, conf in tag_confidence.items()])
            metadata.at[index, "RMS"] = rms
            metadata.at[index, "Tempo"] = tempo
            metadata.at[index, "Sleep_Music"] = sleep_music_status
            metadata.at[index, "Emotion_Tag"] = emotion_tag

            print(f"{audio_file}: 태깅 완료 - {tags}, 수면음악: {sleep_music_status}, 감정: {emotion_tag}")
        except Exception as e:
            print(f"처리 실패: {audio_file}, 오류: {e}")

    # 확장된 메타데이터 저장
    try:
        metadata.to_csv(output_metadata_file, index=False)
        print(f"확장된 메타데이터가 저장되었습니다: {output_metadata_file}")
    except Exception as e:
        print(f"메타데이터 저장 실패: {output_metadata_file}, 오류: {e}")

# 실행
update_metadata_with_features_and_tags(AUDIO_DIR, METADATA_FILE, OUTPUT_METADATA_FILE)

메타데이터 로드 성공: /content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/processed_metadata.csv
처리 중인 파일 경로: /content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.wav
Librosa 파일 로드 성공: /content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.wav, 샘플링 레이트: 16000
Musicnn 태깅 실패: /content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.wav, 오류: `batch_normalization` is not available with Keras 3.
처리 중인 파일 경로: /content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.wav
Librosa 파일 로드 성공: /content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.wav, 샘플링 레이트: 16000
Musicnn 태깅 실패: /con

KeyboardInterrupt: 

In [2]:
!pip install openl3

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached resampy-0.2.2.tar.gz (323 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 41.0 MB/s eta 0:00:00
  Created wheel for openl3: filename=openl3-0.4.2-py2.py3-none-any.whl size=249327028 sha256=33b131cc3dafe8b230a91ba5cf727052f7233b1bd9a7573af1e73db4ed49aad0
  Stored in directory: /root/.cache/pip/wheels/d0/4d/0a/e57b1dc8ead91b3c5709d9de4f02d1cdd3a91f609a8f1c1062
  Created wheel for kapre: filename=kapre-0.3.7-py3-none-any.whl size=29603 sha256=d5ff81d60a956c2177e82d73017faf9a9e2e6190e94e8adb88623f7fc80dac47
  Stored in directory: /root/.cache/pip/wheels/3e/54/f9/37e9f36590a5431dc207f7dc0eb2e607ff8958d5728e45ef58
  Created wheel for resampy: filename=resampy-0.2.2-py3-non

In [2]:
!pip uninstall -y numpy pandas librosa openl3
!pip install numpy==1.19.5  # np.complex를 지원하는 마지막 버전
!pip install librosa==0.8.1
!pip install pandas
!pip install openl3


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: librosa 0.8.1
Uninstalling librosa-0.8.1:
  Successfully uninstalled librosa-0.8.1
Found existing installation: openl3 0.4.2
Uninstalling openl3-0.4.2:
  Successfully uninstalled openl3-0.4.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 41.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: ERROR: Failed to build instal

In [4]:
!pip install --upgrade pip setuptools wheel

  Using cached setuptools-75.6.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.0 MB/s eta 0:00:00
Using cached setuptools-75.6.0-py3-none-any.whl (1.2 MB)
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
  Attempting uninstall: wheel
    Found existing installation: wheel 0.45.0
    Uninstalling wheel-0.45.0:
      Successfully uninstalled wheel-0.45.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is

In [1]:
!pip install numpy==1.19.5 --no-deps
!pip install pandas==1.1.5 --no-deps
!pip install librosa==0.8.1 --no-deps
!pip install openl3 --no-deps


  Using cached numpy-1.19.5.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (numpy)
  Using cached pandas-1.1.5.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pandas (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem

In [2]:
import numpy
import pandas
import librosa
import openl3

print("Numpy 버전:", numpy.__version__)
print("Pandas 버전:", pandas.__version__)
print("Librosa 버전:", librosa.__version__)
print("OpenL3 버전:", openl3.__version__)



AttributeError: module 'numpy' has no attribute 'complex'.
`np.complex` was a deprecated alias for the builtin `complex`. To avoid this error in existing code, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [3]:
import os
import pandas as pd
import librosa
import openl3

# 경로 설정
AUDIO_DIR = "/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen"
METADATA_FILE = "/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/processed_metadata.csv"
OUTPUT_METADATA_FILE = "/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/extended_metadata.csv"

# OpenL3를 사용한 특징 추출
def extract_openl3_features(audio_file):
    """
    OpenL3를 사용하여 오디오 파일의 임베딩을 추출합니다.
    """
    try:
        y, sr = librosa.load(audio_file, sr=None, mono=True)  # 단일 채널로 변환
        emb, ts = openl3.get_audio_embedding(y, sr, content_type="music", embedding_size=512)
        return emb.mean(axis=0)  # 평균 임베딩 반환
    except Exception as e:
        print(f"OpenL3 특징 추출 실패: {audio_file}, 오류: {e}")
        return None

# RMS/BPM 계산
def calculate_audio_features(audio_file):
    """
    오디오 파일에서 RMS(음량)와 BPM(템포)을 계산합니다.
    """
    y, sr = librosa.load(audio_file, sr=None)
    rms = librosa.feature.rms(y=y).mean()
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    return rms, tempo

# 수면음악 조건 분류
def classify_sleep_music(rms, tempo):
    """
    RMS와 템포를 기반으로 수면음악 여부를 분류합니다.
    """
    is_loud = rms > 0.05  # 예: RMS 0.05 이상은 시끄럽다고 판단
    is_fast = tempo > 90  # 예: BPM 90 이상은 빠른 템포로 간주
    if is_loud or is_fast:
        return "수면음악 아님"
    return "수면음악"

# 메타데이터 업데이트 함수
def update_metadata_with_features(audio_dir, metadata_file, output_metadata_file):
    """
    기존 메타데이터에 OpenL3 임베딩 및 추가 특성을 포함한 정보를 추가합니다.
    """
    # 메타데이터 읽기
    try:
        metadata = pd.read_csv(metadata_file)
        print(f"메타데이터 로드 성공: {metadata_file}")
    except Exception as e:
        print(f"메타데이터 로드 실패: {metadata_file}, 오류: {e}")
        return

    # 새로운 컬럼 추가
    metadata["OpenL3_Features"] = ""
    metadata["RMS"] = ""
    metadata["Tempo"] = ""
    metadata["Sleep_Music"] = ""

    # 각 오디오 파일 처리
    for index, row in metadata.iterrows():
        try:
            # 오디오 파일 경로 생성
            audio_file = os.path.join(audio_dir, row['audio'])  # 'audio' 컬럼 확인
            print(f"처리 중인 파일 경로: {audio_file}")

            if not os.path.exists(audio_file):
                print(f"파일이 존재하지 않습니다: {audio_file}")
                continue

            # OpenL3 특징 추출
            openl3_features = extract_openl3_features(audio_file)
            if openl3_features is None:
                continue

            # RMS/BPM 계산
            rms, tempo = calculate_audio_features(audio_file)

            # 수면 음악 분류
            sleep_music_status = classify_sleep_music(rms, tempo)

            # 메타데이터 업데이트
            metadata.at[index, "OpenL3_Features"] = ",".join(map(str, openl3_features))
            metadata.at[index, "RMS"] = rms
            metadata.at[index, "Tempo"] = tempo
            metadata.at[index, "Sleep_Music"] = sleep_music_status

            print(f"{audio_file}: OpenL3 특징 추출 완료, 수면음악: {sleep_music_status}")
        except Exception as e:
            print(f"처리 실패: {audio_file}, 오류: {e}")

    # 확장된 메타데이터 저장
    try:
        metadata.to_csv(output_metadata_file, index=False)
        print(f"확장된 메타데이터가 저장되었습니다: {output_metadata_file}")
    except Exception as e:
        print(f"메타데이터 저장 실패: {output_metadata_file}, 오류: {e}")

# 실행
update_metadata_with_features(AUDIO_DIR, METADATA_FILE, OUTPUT_METADATA_FILE)


AttributeError: module 'numpy' has no attribute 'complex'.
`np.complex` was a deprecated alias for the builtin `complex`. To avoid this error in existing code, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [11]:
ㅛmetadata = "/content/drive/MyDrive/프로젝트/케어크루즈 인턴/data/maestro_v3/processed_musicgen/processed_metadata.csv"

for index, row in metadata.iterrows():
    audio_file = os.path.join(audio_dir, row['audio'])
    if not os.path.exists(audio_file):
        print(f"파일이 존재하지 않습니다: {audio_file}")
        continue

    try:
        # Librosa 파일 로드 테스트
        y, sr = librosa.load(audio_file, sr=None)
        print(f"Librosa 파일 로드 성공: {audio_file}, 샘플링 레이트: {sr}")

        # Musicnn 태깅 테스트
        tags, tag_confidence = extractor(audio_file, model='MTT_musicnn')
        print(f"Musicnn 태깅 성공: {audio_file}, 태그: {tags}")

    except Exception as e:
        print(f"처리 실패: {audio_file}, 오류: {e}")

AttributeError: 'str' object has no attribute 'iterrows'

In [ ]:
from musicnn.extractor import extractor
import librosa

def extract_features(file_path):
    tags, _ = extractor(file_path, model='MSD_musicnn', input_length=3.0)
    y, sr = librosa.load(file_path, sr=None)
    rms = librosa.feature.rms(y=y).mean()
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

    loudness_tag = "조용함" if rms < 0.02 else "보통" if rms < 0.1 else "시끄러움"
    return {
        "tags": tags,
        "rms": rms,
        "tempo": tempo,
        "loudness_tag": loudness_tag
    }

def classify_sleep_music(features):
    is_loud = features["rms"] > 0.05
    is_fast = features["tempo"] > 90

    tags = features["tags"]
    tags.append("수면음악 아님" if is_loud or is_fast else "수면음악")
    if is_loud:
        tags.append("시끄러움")
    if is_fast:
        tags.append("빠름")
    return tags

def map_emotion_to_tags(tags):
    calm_tags = {"relaxing", "calm", "peaceful"}
    energetic_tags = {"exciting", "energetic", "intense"}

    emotion = "neutral"
    if calm_tags.intersection(tags):
        emotion = "calm"
    elif energetic_tags.intersection(tags):
        emotion = "energetic"

    tags.append(f"감정: {emotion}")
    return tags

# 전체 파이프라인
file_path = "example_piano.wav"
features = extract_features(file_path)
tags = classify_sleep_music(features)
tags = map_emotion_to_tags(tags)

print(f"최종 태그: {tags}")